# PART 06 파인 튜닝과 모델의 실전 활용

### chatgpt.env 환경파일 준비
 * 일반적으로 환경 변수는 .env 파일에 저장되지만, 구글 코랩 사용자의 편의를 위해 이 책에서는 chatgpt.env를 사용합니다.
 * 실제 개발 환경에서는 보통 .env를 사용하니, 이 점을 기억해 두세요.

### 기본 환경 설정
### chatgpt.env 환경파일 준비
 * 일반적으로 환경 변수는 .env 파일에 저장되지만, 구글 코랩 사용자의 편의를 위해 이 책에서는 chatgpt.env를 사용합니다.
 * 실제 개발 환경에서는 보통 .env를 사용하니, 이 점을 기억해 두세요.


 ### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치
    * 라이브러리 불일치로 인한 에러 발생시, 추가 라이브러리 설치 필요.
    * 에러 : TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [1]:
!pip install openai
!pip install httpx==0.27.2

In [3]:
from openai import OpenAI
import os

# chatgpt.env 파일에서 환경 변수 읽기
with open("chatgpt.env") as env:
    for line in env:
        key, value = line.strip().split("=")
        os.environ[key] = value

# OpenAI 클라이언트 초기화
client = OpenAI( api_key=os.environ['API_KEY'] )

## 19장 파인 튜닝의 이해 및 구현

### 19-1. 파인 튜닝의 실제 적용

* data.jsonl

```
{"messages": [{"role": "system", "content": "You are a smart home assistant"}, {"role": "user", "content": "When do I have to start the heater?"}, {"role": "assistant","content": "Every day in the morning at 7AM. You should stop it at 2PM"}]}
{"messages": [{"role": "system", "content": "You are a smart home assistant."}, {"role": "user", "content": "Where is the garage remote control?"}, {"role": "assistant", "content": "Next to the yellow door, on the key ring"}]}
{"messages": [{"role": "system", "content": "You are a smart home assistant."}, {"role": "user", "content": "Is it necessary to program the scent diffuser every day?"},{"role": "assistant", "content": "The scent diffuser is already programmed, you just need to recharge it when its battery is low"}]}
{"messages": [{"role": "system", "content": "You are a smart home assistant."}, {"role": "user", "content": "How do I reset the WiFi router?"}, {"role": "assistant", "content": "Unplug the router, wait for 30 seconds, then plug it back in."}]}
{"messages": [{"role": "system", "content": "You are a smart home assistant."}, {"role": "user", "content": "How do I connect my phone to the smart TV?"}, {"role": "assistant", "content": "Go to settings on your phone, select 'Screen Mirroring', and choose the smart TV from the list."}]}
{"messages": [{"role": "system", "content": "You are a smart home assistant."}, {"role": "user", "content": "What's the Wi-Fi password?"}, {"role": "assistant", "content": "The Wi-Fi password is 'HomeNetwork123'."}]}
{"messages": [{"role": "system", "content": "You are a smart home assistant."}, {"role": "user", "content": "How to hard-reset the smart TV?"}, {"role": "assistant", "content": "Press and hold the power button on the remote for 10 seconds."}]}
{"messages": [{"role": "system", "content": "You are a smart home assistant."}, {"role": "user", "content": "How to call the police?"}, {"role": "assistant", "content":"Dial 911 on your phone."}]}
{"messages": [{"role": "system", "content": "You are a smart home assistant."}, {"role": "user", "content": "How to change the wifi password?"}, {"role": "assistant", "content": "Go to the router settings, select 'Wireless', and change the password."}]}
{"messages": [{"role": "system", "content": "You are a smart home assistant."}, {"role": "user", "content": "How to change the channel on the TV?"}, {"role": "assistant", "content": "Press the 'Channel Up' or 'Channel Down' button on the remote."}]}
```

```
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "히터를 언제 켜야 하나요?"}, {"role": "assistant", "content": "매일 아침 7시에 켜세요. 오후 2시에 꺼야 합니다."}]}
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "차고 리모컨이 어디에 있나요?"}, {"role": "assistant", "content": "노란색 문 옆, 열쇠 고리에 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "향기 디퓨저를 매일 프로그래밍해야 하나요?"}, {"role": "assistant", "content": "향기 디퓨저는 이미 프로그래밍되어 있으므로 배터리가 낮을 때 충전만 하면 됩니다."}]}
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "WiFi 라우터를 어떻게 재설정하나요?"}, {"role": "assistant", "content": "라우터의 전원을 뽑고 30초 동안 기다린 후 다시 연결하세요."}]}
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "내 휴대폰을 스마트 TV에 어떻게 연결하나요?"}, {"role": "assistant", "content": "휴대폰 설정에서 '스크린 미러링'을 선택한 후 목록에서 스마트 TV를 선택하세요."}]}
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "와이파이 비밀번호가 뭐야?"}, {"role": "assistant", "content": "와이파이 비밀번호는 'HomeNetwork123'입니다."}]}
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "스마트 TV를 하드 리셋하는 방법은?"}, {"role": "assistant", "content": "리모컨의 전원 버튼을 10초 동안 누르고 계세요."}]}
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "경찰에 어떻게 전화하나요?"}, {"role": "assistant", "content": "휴대폰에서 911에 전화하세요."}]}
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "와이파이 비밀번호를 어떻게 변경하나요?"}, {"role": "assistant", "content": "라우터 설정으로 이동하여 '무선'을 선택한 후 비밀번호를 변경하세요."}]}
{"messages": [{"role": "system", "content": "당신은 스마트 홈 어시스턴트입니다."}, {"role": "user", "content": "TV 채널을 어떻게 변경하나요?"}, {"role": "assistant", "content": "리모컨에서 '채널 업' 또는 '채널 다운' 버튼을 누르세요."}]}
```

In [ ]:
### fine_tuning.py

In [ ]:
import os, sys  # os와 sys 모듈 불러오기

# 파일 경로 정의
file_path = os.path.join('data.jsonl')  # 업로드할 파일 이름

# 파일 업로드
uploaded = client.files.create(
    file=open(file_path, "rb"),  # 파일을 읽기 모드(rb)로 열어 업로드
    purpose="fine-tune"  # 목적을 파인 튜닝으로 설정
)

# 업로드된 파일 ID 가져오기
file_id = uploaded.id
print(file_id)


file-TDD2E7WUk5AGnYKuwYuSSs


### 19-1 파인 튜닝의 실제 적용

In [ ]:
import os, sys  # os와 sys 모듈 불러오기

# 파일 경로 정의
file_path = os.path.join('data.jsonl')  # 업로드할 파일 이름

# 파일 업로드
uploaded = client.files.create(
    file=open(file_path, "rb"),  # 파일을 읽기 모드(rb)로 열어 업로드
    purpose="fine-tune"  # 목적을 파인 튜닝으로 설정
)

# 업로드된 파일 ID 가져오기
file_id = uploaded.id
print(file_id)


file-K7GVPhYsw8NyEKcoATzZHt


In [ ]:
# 사용할 모델 설정
model = "gpt-4o-mini-2024-07-18"

# 파인 튜닝 작업 생성
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=file_id,  # 업로드된 파일 ID를 학습 데이터로 사용
    model=model  # 사용할 모델 지정
)

print()
print("파일 유효성 검사 중입니다.")  # 파일 유효성 검사 중 메시지 출력

# 파일 유효성 검사 진행 확인
while fine_tune_job.status == "validating_files":
    fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tune_job.id)  # 현재 작업 상태를 다시 불러옴
    print(".", end="", flush=True)  # 진행 표시 출력

print("파인 튜닝이 진행 중입니다.")  # 파인 튜닝 진행 중 메시지 출력
print("파인 튜닝 상태", fine_tune_job.status)

# 파인 튜닝이 실행 중이거나 대기 중일 때 작업 상태 확인
while fine_tune_job.status == "running" or fine_tune_job.status == "queued":
    fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tune_job.id)  # 현재 작업 상태를 다시 불러옴
    print(".", end="", flush=True)   # 진행 표시 출력

print()
print("파인 튜닝이 완료되었습니다.")  # 파인 튜닝 완료 메시지 출력

# 새로운 모델 이름 출력
print("새로운 모델 이름: " + fine_tune_job.fine_tuned_model)



파일 유효성 검사 중입니다.
................................................................................................................................파인 튜닝이 진행 중입니다.
파인 튜닝 상태 queued
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
# 새로운 모델 이름 출력
print("파인 튜닝 작업 status: " + fine_tune_job.status)

파인 튜닝 작업 status: succeeded


### 파인튜닝 실습

In [ ]:
!pip install colorama

In [ ]:
# 모델 이름 설정
# model = "$FINE_TUNED_MODEL"
model = "ft:gpt-4o-mini-2024-07-18:personal::AEFlKEZh"

# 기본 시스템 메시지 설정
base_messages = [
    {
        "role": "system",
        "content": "당신은 스마트 홈 어시스턴트입니다."  # 스마트 홈 역할 정의
    }
]

# 사용자 입력을 계속 받기 위한 무한 반복문
while True:
    messages = base_messages.copy()  # 기본 메시지를 복사하여 새로운 메시지 리스트 생성

    # 사용자 입력 받기
    request = input("Input: (종료하려면 'exit' 또는 'quit'을 입력하세요.): ")

    # 'exit' 또는 'quit' 입력 시 반복문 종료
    if request.lower() in ["exit", "quit"]:
        break

    # 사용자 입력을 메시지 리스트에 추가
    messages.append(
        {
            "role": "user",
            "content": f"{request}"  # 사용자 입력을 content로 추가
        }
    )

    # OpenAI API에 대화 생성 요청 보내기
    response = client.chat.completions.create(
        model=model,        # 사용할 모델 설정
        messages=messages,  # 메시지 리스트 전달
        max_tokens=200,     # 최대 토큰 수 설정
        temperature=0,      # 응답의 무작위성 설정 (0으로 설정. 일관성 있는 응답 생성)
    )

    # API 응답 내용 추출
    content = response.choices[0].message.content.strip()  # 응답 내용에서 공백 제거

    # 간단히 출력
    print(f"Output: {content}")
    print()  # 출력 후 줄바꿈


Input: (종료하려면 'exit' 또는 'quit'을 입력하세요.): 와이파이 비밀번호 변경은?
Output: 라우터 설정으로 이동하여 '무선'을 선택한 후 비밀번호를 변경하세요.

Input: (종료하려면 'exit' 또는 'quit'을 입력하세요.): TV 채널 변경
Output: 리모컨에서 '채널 업' 또는 '채널 다운' 버튼을 누르세요.

Input: (종료하려면 'exit' 또는 'quit'을 입력하세요.): exit


In [ ]:
import click  # click 라이브러리 불러오기

# 모델 이름 설정
# model = "$FINE_TUNED_MODEL"
model = "ft:gpt-4o-mini-2024-07-18:personal::AEFlKEZh"

# 기본 시스템 메시지 설정
base_messages = [
    {
        "role": "system",
        "content": "당신은 스마트 홈 어시스턴트입니다."  # 스마트 홈 역할 정의
    }
]

# 사용자 입력을 계속 받기 위한 무한 반복문
while True:
    messages = base_messages.copy()  # 기본 메시지를 복사하여 새로운 메시지 리스트 생성

    # 사용자 입력 받기
    request = input(
        click.style(  # 입력 안내 스타일 지정
            "Input: (종료하려면 'exit' 또는 'quit' 을 입력하세요. ): ",
            fg="green"  # 녹색 텍스트로 프롬프트 표시
        )
    )

    # 'exit' 또는 'quit' 입력 시 반복문 종료
    if request.lower() in ["exit", "quit"]:
        break

    # 사용자 입력을 메시지 리스트에 추가
    messages.append(
        {
            "role": "user",
            "content": f"{request}"  # 사용자 입력을 content로 추가
        }
    )

    # OpenAI API에 대화 생성 요청 보내기
    response = client.chat.completions.create(
        model=model,        # 사용할 모델 설정
        messages=messages,  # 메시지 리스트 전달
        max_tokens=200,     # 최대 토큰 수 설정
        temperature=0,      # 응답의 무작위성 설정 (0으로 설정. 일관성 있는 응답 생성)
    )

    # API 응답 내용 추출
    content = response.choices[0].message.content.strip()  # 응답 내용에서 공백 제거

    # 스타일을 지정하여 출력
    click.echo(
        click.style("Output: ", fg="yellow") + content  # 'Output:' 텍스트를 노란색으로 출력 후 응답 내용 출력
    )
    click.echo()  # 출력 후 줄바꿈

Input: (종료하려면 'exit' 또는 'quit' 을 입력하세요. ): 와이파이 비밀번호 변경은?
Output: 라우터 설정으로 이동하여 '무선'을 선택한 후 비밀번호를 변경하세요.

Input: (종료하려면 'exit' 또는 'quit' 을 입력하세요. ): TV 채널 변경 
Output: 리모컨에서 '채널 업' 또는 '채널 다운' 버튼을 누르세요.

Input: (종료하려면 'exit' 또는 'quit' 을 입력하세요. ): quit


### 19-2. 파인 튜닝 최적화: 학습률, 배치 크기, 에포크 설정 가이드

### Fine-Tuning Best Practices - p185
 * data_analysis.py

In [ ]:
import json
from collections import defaultdict
import os

# 데이터 파일 경로 설정
file_path = os.path.join('data.jsonl')  # 데이터 파일 이름

# 데이터셋 파일 형식 검증
with open(file_path, 'r', encoding='utf-8') as f:
    try:
        # 파일을 읽어 각 줄을 JSON 형식으로 파싱
        dataset = [json.loads(line) for line in f]
    except:
        # 파일 형식이 올바르지 않으면 오류 발생
        raise ValueError("데이터셋은 유효한 JSONL 파일이어야 합니다.")

# 데이터셋 크기 검증
size = len(dataset)
if size < 10:
    # 최소 10개 이상의 예제가 있어야 함
    raise ValueError("데이터셋에는 최소 10개의 예제가 포함되어야 합니다.")

# 데이터셋 내용 검증
format_errors = defaultdict(int)  # 형식 오류를 저장할 딕셔너리 초기화
for line in dataset:
    # 데이터 유형 확인:
    # 각 항목이 딕셔너리인지 검증
    if not isinstance(line, dict):
        format_errors["data_type"] += 1
        continue

    # 메시지 리스트 존재 여부 확인:
    # 'messages' 리스트가 포함되어 있는지 확인
    messages = line.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        # 메시지 키 확인:
        # 각 메시지에 'role' 및 'content' 키가 있는지 확인
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        # 메시지에 포함될 수 있는 유효한 키
        valid_keys = ("role", "content", "name", "function_call")

        # 인식되지 않은 키 확인:
        # 유효한 키 목록에 없는 키가 포함되어 있는지 확인
        if any(k not in valid_keys for k in message):
            format_errors["message_unrecognized_key"] += 1

        # 메시지에 허용되는 유효한 역할(role) 목록
        valid_roles = ("system", "user", "assistant", "function")

        # 역할(role) 확인:
        # 'role' 값이 유효한 역할 중 하나인지 검증
        if message.get("role", None) not in valid_roles:
            format_errors["unrecognized_role"] += 1

        # 'content'와 'function_call' 값 가져오기
        content = message.get("content", None)
        function_call = message.get("function_call", None)

        # 콘텐츠(content) 확인:
        # 'content'가 텍스트 형태의 문자열인지, 또는 'function_call'이 있는지 검증
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    # assistant 메시지 존재 여부 확인:
    # 각 대화에 최소 한 개 이상의 assistant 메시지가 포함되어 있는지 확인
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

# 오류가 있는 경우, 오류 내용 출력
if format_errors:
    print("발견된 오류:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
    raise ValueError("데이터셋에 오류가 포함되어 있습니다.")


### fine_tuning.py

In [ ]:
# fine_tuning.py

# 인증 부분은 구글 코랩의 경우, 앞에서 수행되었다면, 생략이 가능합니다.
# 인증 부분
# init_api()

import os, sys, json
from collections import defaultdict

# 데이터 파일 경로 설정
file_path = os.path.join(
    os.getcwd(),  # 현재 파일의 디렉터리 경로
    'data_old.jsonl'  # 데이터 파일 이름
)

# 데이터셋 파일 형식 검증
with open(file_path, 'r', encoding='utf-8') as f:
    try:
        # 각 줄을 JSON 형식으로 로드하여 리스트로 저장
        dataset = [json.loads(line) for line in f]
    except:
        # JSONL 파일 형식이 유효하지 않으면 오류 발생
        raise ValueError("데이터셋은 유효한 JSONL 파일이어야 합니다.")

# 데이터셋 크기 검증
size = len(dataset)
if size < 10:
    # 최소 10개 이상의 예제가 있어야 함
    raise ValueError("데이터셋에는 최소 10개의 예제가 포함되어야 합니다.")

# 데이터셋 내용 검증
format_errors = defaultdict(int)  # 형식 오류를 저장할 딕셔너리 초기화
for line in dataset:
    # 데이터 유형 확인:
    # 각 항목이 딕셔너리인지 검증
    if not isinstance(line, dict):
        format_errors["data_type"] += 1
        continue

    # 메시지 리스트 존재 여부 확인:
    # 'messages' 리스트가 포함되어 있는지 확인
    messages = line.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        # 메시지 키 확인:
        # 각 메시지에 'role' 및 'content' 키가 있는지 확인
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        # 메시지에 포함될 수 있는 유효한 키
        valid_keys = ("role", "content", "name", "function_call")

        # 인식되지 않은 키 확인:
        # 유효한 키 목록에 없는 키가 포함되어 있는지 확인
        if any(k not in valid_keys for k in message):
            format_errors["message_unrecognized_key"] += 1

        # 메시지에 허용되는 유효한 역할(role) 목록
        valid_roles = ("system", "user", "assistant", "function")

        # 역할(role) 확인:
        # 'role' 값이 유효한 역할 중 하나인지 검증
        if message.get("role", None) not in valid_roles:
            format_errors["unrecognized_role"] += 1

        # 'content'와 'function_call' 값 가져오기
        content = message.get("content", None)
        function_call = message.get("function_call", None)

        # 콘텐츠(content) 확인:
        # 'content'가 텍스트 형태의 문자열인지, 또는 'function_call'이 있는지 검증
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    # assistant 메시지 존재 여부 확인:
    # 각 대화에 최소 한 개 이상의 assistant 메시지가 포함되어 있는지 확인
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

# 오류가 있는 경우, 오류 내용 출력
if format_errors:
    print("발견된 오류:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
    raise ValueError("데이터셋에 오류가 포함되어 있습니다.")

# 파일 업로드
uploaded = client.files.create(
    file=open(file_path, "rb"),  # 파일을 읽기 모드로 열어 업로드
    purpose="fine-tune"          # 파인 튜닝 용도로 파일 설정
)
file_id = uploaded.id  # 업로드된 파일의 ID 저장
model = "gpt-4o-mini-2024-07-18"  # 파인 튜닝할 모델 설정

# 파인 튜닝 작업 생성
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=file_id,
    model=model,
)

print("\n파일 검증 진행 중")
while fine_tune_job.status == "validating_files":  # 파일 검증 중일 때 상태 체크
    fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
    print(".", end="", flush=True)  # 진행 상황 표시

print("\n파인 튜닝 진행 중")
# 파인 튜닝 실행 중 또는 대기 중일 때 상태 체크
while fine_tune_job.status == "running" or fine_tune_job.status == "queued":
    fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
    print(".", end="", flush=True)  # 진행 상황 표시

print("\n파인 튜닝 완료")
print("새로운 모델의 이름은: " + fine_tune_job.fine_tuned_model)


파일 검증 진행 중
.......................................................................................................................................
파인 튜닝 진행 중
.........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

### 19-2 파인 튜닝 최적화: 학습률, 배치 크기

In [ ]:
from datetime import datetime, timezone
import pytz

def format_timestamp_to_kst(timestamp):
    """
    타임스탬프를 KST (한국 시간대) 시간 문자열로 변환, None이면 None을 반환
    """
    if timestamp is not None:
        utc_time = datetime.fromtimestamp(timestamp, tz=timezone.utc)
        kst_time = utc_time.astimezone(pytz.timezone("Asia/Seoul"))
        return kst_time.strftime('%Y-%m-%d %H:%M:%S')
    return None

# 테스트
print(f"created_at=1701832706 => created_at : {format_timestamp_to_kst(1701832706)}")


created_at=1701832706 => created_at : 2023-12-06 12:18:26


In [ ]:
models = client.fine_tuning.jobs.list()

print("파인 튜닝 작업 리스트 개수 : ", len(models.data))

# 리스트를 확인합니다.
for model in models.data:
    print(model)
    created_time = format_timestamp_to_kst(model.created_at)
    finished_time = format_timestamp_to_kst(model.finished_at)
    print(f"id : {model.id}, fine_tuned_model : {model.fine_tuned_model}")
    print(f"생성일시 : {created_time}, 완료일시 : {finished_time}")
    print()


파인 튜닝 작업 리스트 개수 :  0


In [ ]:
# analytics.py

## 초기 인증 함수 불러오기
## init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"))

# 파인튜닝 작업 가져오기. 확인한 작업 ID를 지정합니다.
job_id = "ftjob-hNsa27PCpGvCqUXA54IM2BsV"


fine_tune_job = client.fine_tuning.jobs.retrieve(job_id)

# 작업 상태 확인
status = fine_tune_job.status

# 작업이 성공적으로 완료되었을 경우
if status == "succeeded":
    # 학습한 에포크 수 출력
    print(f"Number of epochs: {fine_tune_job.hyperparameters.n_epochs}")

    # 학습률 배수 출력
    print(f"Learning rate multiplier: \
{fine_tune_job.hyperparameters.learning_rate_multiplier}")

    # 배치 크기 출력
    print(f"Batch size: {fine_tune_job.hyperparameters.batch_size}")

    # 학습한 토큰 수 출력
    print(f"Trained tokens: {fine_tune_job.trained_tokens}")

    # 학습 손실 값 출력
    print("Training loss:")

    # 작업 이벤트 목록 가져오기
    events = client.fine_tuning.jobs.list_events(job_id)

    # 이벤트 데이터에서 손실 값 출력
    for event in events.data:
        if event.type == 'metrics':
            # 딕셔너리 키를 사용하여 값 접근
            step = event.data['step']
            train_loss = event.data['train_loss']
            print(f"Step {step}: training loss={train_loss}")
else:
    print("작업이 아직 진행 중이거나 실패했습니다.")


Number of epochs: 10
Learning rate multiplier: 1.8
Batch size: 1
Trained tokens: 5490
Training loss:
Step 100: training loss=0.0002349506685277447
Step 99: training loss=0.00010747175838332623
Step 98: training loss=0.0006081616156734526
Step 97: training loss=0.00013605754065793008
Step 96: training loss=0.00013368026702664793
Step 95: training loss=0.00030587849323637784
Step 94: training loss=2.1696090698242188e-05
Step 93: training loss=0.0003930005186703056
Step 92: training loss=0.000561833381652832
Step 91: training loss=0.5122135281562805
Step 90: training loss=0.0003412146179471165
Step 89: training loss=0.00018758358783088624
Step 88: training loss=0.00017811701400205493
Step 87: training loss=0.00014527638268191367
Step 86: training loss=0.00037149948184378445


### 모델 평가

## 20장 파인 튜닝 고급 예제 – 정신 건강 코치

In [4]:
# Create the folder (만들어져 있으면 다시 실행할 필요 없음)
!rm -rf data
!mkdir -p data

# Download the data
!wget https://raw.githubusercontent.com/LDJWJ/OpenAIGPTForPythonDevelopersFiles/main/datasets/ch20_mental_health_counseling/data.json -O data/data.json

--2024-12-19 13:58:29--  https://raw.githubusercontent.com/LDJWJ/OpenAIGPTForPythonDevelopersFiles/main/datasets/ch20_mental_health_counseling/data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4790520 (4.6M) [text/plain]
Saving to: ‘data/data.json’

data/data.json      100%[===================>]   4.57M  --.-KB/s    in 0.02s   

2024-12-19 13:58:30 (196 MB/s) - ‘data/data.json’ saved [4790520/4790520]



### 20-2. 데이터 셋 준비하기

* 아래 설치 시, 정상적으로 설치가 완료가 안될 경우, 다시 한번 실행하면 설치는 되는 경우가 있습니다.(설치 시점의 차이로 인해 발생되는 것으로 보임)

In [1]:
!pip install setuptools==69.0.3
!pip install langdetect==1.0.9
!pip install unidecode==1.3.8


  Using cached langdetect-1.0.9-py3-none-any.whl


In [2]:
!pip install openai
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


### 세션 시작 후, OpenAI API 초기화 및 인증

In [4]:
import os
from openai import OpenAI

with open("chatgpt.env") as env:
  for line in env:
    key, value = line.strip().split("=")
    os.environ[key] = value

client = OpenAI(api_key  = os.environ['API_KEY'])

In [5]:
# prepare_data.py
import json, re
from langdetect import detect
from unidecode import unidecode

# 데이터 리스트 초기화
data = []

# 텍스트 정제 함수 정의
def clean_text(text):
    # 일반적인 유니코드 문자를 ASCII 문자로 변환
    text = unidecode(text)
    # URL 제거
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # 구두점 주변의 공백 수정
    text = re.sub(r'\s*([,.!?])\s*', r'\1 ', text)
    # 앞뒤 공백 제거
    text = text.strip()
    # 구두점 뒤에 공백이 없을 경우 추가
    text = re.sub(r'([:,.!?])([^\s])', r'\1 \2', text)
    return text

# 기존의 'data.jsonl' 파일 내용 초기화 (모든 데이터 삭제)
open('data/data.jsonl', 'w').close()

# 시스템 메시지 프롬프트 정의
system_prompt = (
    "당신은 MendMind, AI 멘탈 헬스 코치입니다. "
    "당신의 목적은 사용자가 정신 건강 여정을 탐색하는 과정에서 "
    "공감, 이해, 정서적·심리적 도전 과제에 대한 통찰력을 제공하는 것입니다. "
    "일반적인 조언과 정서적 지원은 제공할 수 있지만, "
    "개인적인 연락을 하거나, 일정 예약을 하거나, 특정 위치 정보를 공유할 수 없습니다. "
    "당신의 역할은 사용자가 대처 전략을 찾도록 돕고, 정신 건강 주제에 대한 정보를 제공하며, "
    "필요 시 전문가 리소스를 안내하는 것입니다. "
    "일반적인 대화를 나눌 수 있지만, 주요 초점은 비밀을 유지하면서 사용자에게 최선의 지원을 제공하는 것입니다."
)

# 데이터 파일 열기 및 데이터 읽기
with open('data/data.json', 'r') as file:
    for line in file:
        json_line = json.loads(line)  # 각 줄의 JSON 데이터를 파싱
        context = json_line["Context"]  # 사용자 질문 (Context) 추출
        response = json_line["Response"]  # 심리학자의 답변 (Response) 추출
        try:
            # Context가 비어있지 않고 Response가 10단어 이상일 때
            if len(context) > 0 and len(response.split()) > 10:
                # Context와 Response가 모두 영어일 경우
                if detect(context) == "en" and detect(response) == "en":
                    # 시스템, 사용자, 어시스턴트 메시지 구성
                    system = {
                        "role": "system",
                        "content": system_prompt
                    }
                    user = {
                        "role": "user",
                        "content": clean_text(context)
                    }
                    assistant = {
                        "role": "assistant",
                        "content": clean_text(response)
                    }
                    messages = {
                        "messages": [system, user, assistant]
                    }
                    # data.jsonl 파일에 메시지 작성 (추가)
                    with open('data/data.jsonl', 'a') as file:
                        file.write(json.dumps(messages) + '\n')

        except Exception as e:
            # 오류 발생 시 오류 메시지 출력 및 해당 데이터 출력
            print(f"Error:\n Context: {context}\n Response: {response}")


### 데이터 검증 및 변환 후, 파인 튜닝 작업 시작

In [ ]:
# fine_tuning.py

import os, sys, json
from collections import defaultdict

# 실행 환경이 구글 코랩의 경우, 인증 부분은 앞에서 실행했다면, 작성하지 않아도 됩니다.
## 초기 API KEY 가져와 OpenAI 초기화
## init_api()
from openai import OpenAI

client = OpenAI(api_key  = os.environ.get("API_KEY"))


# 데이터 파일 경로 설정
file_path = os.path.join('data/data_mini.jsonl')  # 데이터 파일 이름 설정

# 데이터셋 파일 형식 검증
with open(file_path, 'r', encoding='utf-8') as f:
    try:
        dataset = [json.loads(line) for line in f]  # 파일에서 각 줄을 읽어 JSON 객체로 변환
    except:
        raise ValueError("데이터셋은 유효한 JSONL 파일이어야 합니다")

# 데이터셋 크기 검증
size = len(dataset)
if size < 10:
    raise ValueError("데이터셋은 최소 10개의 예제를 포함해야 합니다")

# 데이터셋 내용 검증
format_errors = defaultdict(int)  # 형식 오류를 저장할 딕셔너리 초기화
for line in dataset:
    # 데이터 타입 확인:
    # 각 항목이 딕셔너리인지 확인
    if not isinstance(line, dict):
        format_errors["data_type"] += 1
        continue

    # 메시지 목록 존재 여부 확인:
    # 각 항목에 'messages' 리스트가 있는지 확인
    messages = line.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    # 각 메시지의 키와 내용 확인
    for message in messages:
        # 메시지 키 확인:
        # 각 메시지에 'role'과 'content' 키가 있는지 확인
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        # 메시지에서 유효한 키 목록 설정
        valid_keys = (
            "role",          # 역할
            "content",       # 메시지 내용
            "name",          # 이름
            "function_call"  # 함수 호출
        )

        # 메시지에 유효하지 않은 키가 있는지 확인
        if any(k not in valid_keys for k in message):
            format_errors["message_unrecognized_key"] += 1

        # 유효한 역할 목록 설정
        valid_roles = (
            "system",     # 시스템
            "user",       # 사용자
            "assistant",  # 어시스턴트
            "function"    # 함수
        )

        # 역할 값이 유효한지 확인
        if message.get("role", None) not in valid_roles:
            format_errors["unrecognized_role"] += 1

        # 'content' 및 'function_call'의 값 검증
        content = message.get("content", None)
        function_call = message.get("function_call", None)

        # 'content'가 문자열이며 텍스트인지 확인,
# 'content' 또는 'function_call'이 있어야 함
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    # 어시스턴트 메시지 존재 여부 확인:
    # 메시지 중 'assistant' 역할이 있는지 확인
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

# 형식 오류가 있는 경우 출력
if format_errors:
    print("발견된 오류:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
    raise ValueError("데이터셋에 오류가 포함되어 있습니다")

# 파일 업로드
uploaded = client.files.create(
    file=open(file_path, "rb"),  # 데이터 파일을 바이너리 모드로 열기
    purpose="fine-tune"  # 목적을 '파인튜닝'으로 설정
)

file_id = uploaded.id  # 업로드된 파일 ID 가져오기

# 모델 설정 및 파인튜닝 작업 생성
model = "gpt-4o-mini-2024-07-18"
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=file_id,  # 학습 파일 ID 설정
    model=model            # 사용할 모델 설정
)

# 파일 검증 진행 중
print("파일 검증 중")
while fine_tune_job.status == "validating_files":
    fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
    print(".", end="", flush=True)

# 파인튜닝 진행 중
print("\n파인튜닝 진행 중")
while fine_tune_job.status == "running" or fine_tune_job.status == "queued":
    fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
    print(".", end="", flush=True)

# 파인튜닝 완료
print("\n파인튜닝 완료")
print("새로운 모델의 이름은: " + fine_tune_job.fine_tuned_model)


파일 검증 중
......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
파인튜닝 진행 중
...............................................................................................................................................................................................................................................................

### 비용이 부담이 될 경우, 일부 데이터만 학습을 위해 원본 데이터를 1/4로 줄이기

### data.jsonl 이 fine_tuning을 하는데 비용과 시간이 들어 이를 위한 data.jsonl을 1/3의 크기로 만들어, 이를 가지고 fine_tuning을 수행
 * 원본 크기는 data.jsonl은 약 시간 25분, 비용 USD 34.78
 * 따라서 이를 1/3 정도로 줄여서 수행.

### 데이터를 줄이는 코딩

In [6]:
import json

# 원본 data.jsonl 파일 경로
input_file_path = 'data/data.jsonl'

# 줄여서 저장할 파일 경로
output_file_path = 'data/data_mini.jsonl'

# 줄여서 사용할 비율 (1/4)
reduction_factor = 3

def reduce_jsonl_file(input_path, output_path, reduction_factor):
    """
    input_path에서 읽은 data.jsonl 파일을 1/reduction_factor 크기로 줄여서 output_path에 저장
    """
    # 원본 파일을 읽기
    with open(input_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # 전체 라인 중 1/3만큼을 선택
    reduced_lines = lines[::reduction_factor]

    # 새로운 파일에 줄인 데이터를 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        for line in reduced_lines:
            f.write(line)

    print(f"Original file size: {len(lines)} lines")
    print(f"Reduced file size: {len(reduced_lines)} lines")
    print(f"Reduced data saved to {output_path}")

# 함수 호출하여 파일 줄이기
reduce_jsonl_file(input_file_path, output_file_path, reduction_factor)


Original file size: 3433 lines
Reduced file size: 1145 lines
Reduced data saved to data/data_mini.jsonl


### fine_tuning.py
* 1/3 데이터 활용한 파인튜닝
* 위의 코드와 동일하므로, 위의 data.jsonl을 활용하여 확인했다면 추가 실행하지 않아도 됨.

In [10]:
# fine_tuning.py

import os, sys, json
from collections import defaultdict

# 실행 환경이 구글 코랩의 경우, 인증 부분은 앞에서 실행했다면, 작성하지 않아도 됩니다.
## 초기 API KEY 가져와 OpenAI 초기화
## init_api()
from openai import OpenAI

client = OpenAI(api_key  = os.environ.get("API_KEY"))


# 데이터 파일 경로 설정
file_path = os.path.join('data/data_mini.jsonl')  # 데이터 파일 이름 설정

# 데이터셋 파일 형식 검증
with open(file_path, 'r', encoding='utf-8') as f:
    try:
        dataset = [json.loads(line) for line in f]  # 파일에서 각 줄을 읽어 JSON 객체로 변환
    except:
        raise ValueError("데이터셋은 유효한 JSONL 파일이어야 합니다")

# 데이터셋 크기 검증
size = len(dataset)
if size < 10:
    raise ValueError("데이터셋은 최소 10개의 예제를 포함해야 합니다")

# 데이터셋 내용 검증
format_errors = defaultdict(int)  # 형식 오류를 저장할 딕셔너리 초기화
for line in dataset:
    # 데이터 타입 확인:
    # 각 항목이 딕셔너리인지 확인
    if not isinstance(line, dict):
        format_errors["data_type"] += 1
        continue

    # 메시지 목록 존재 여부 확인:
    # 각 항목에 'messages' 리스트가 있는지 확인
    messages = line.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    # 각 메시지의 키와 내용 확인
    for message in messages:
        # 메시지 키 확인:
        # 각 메시지에 'role'과 'content' 키가 있는지 확인
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        # 메시지에서 유효한 키 목록 설정
        valid_keys = (
            "role",          # 역할
            "content",       # 메시지 내용
            "name",          # 이름
            "function_call"  # 함수 호출
        )

        # 메시지에 유효하지 않은 키가 있는지 확인
        if any(k not in valid_keys for k in message):
            format_errors["message_unrecognized_key"] += 1

        # 유효한 역할 목록 설정
        valid_roles = (
            "system",     # 시스템
            "user",       # 사용자
            "assistant",  # 어시스턴트
            "function"    # 함수
        )

        # 역할 값이 유효한지 확인
        if message.get("role", None) not in valid_roles:
            format_errors["unrecognized_role"] += 1

        # 'content' 및 'function_call'의 값 검증
        content = message.get("content", None)
        function_call = message.get("function_call", None)

        # 'content'가 문자열이며 텍스트인지 확인,
# 'content' 또는 'function_call'이 있어야 함
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    # 어시스턴트 메시지 존재 여부 확인:
    # 메시지 중 'assistant' 역할이 있는지 확인
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

# 형식 오류가 있는 경우 출력
if format_errors:
    print("발견된 오류:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
    raise ValueError("데이터셋에 오류가 포함되어 있습니다")

# 파일 업로드
uploaded = client.files.create(
    file=open(file_path, "rb"),  # 데이터 파일을 바이너리 모드로 열기
    purpose="fine-tune"  # 목적을 '파인튜닝'으로 설정
)

file_id = uploaded.id  # 업로드된 파일 ID 가져오기

# 모델 설정 및 파인튜닝 작업 생성
model = "gpt-4o-mini-2024-07-18"
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=file_id,  # 학습 파일 ID 설정
    model=model            # 사용할 모델 설정
)

# 파일 검증 진행 중
print("파일 검증 중")
while fine_tune_job.status == "validating_files":
    fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
    print(".", end="", flush=True)

# 파인튜닝 진행 중
print("\n파인튜닝 진행 중")
while fine_tune_job.status == "running" or fine_tune_job.status == "queued":
    fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
    print(".", end="", flush=True)

# 파인튜닝 완료
print("\n파인튜닝 완료")
print("새로운 모델의 이름은: " + fine_tune_job.fine_tuned_model)

파일 검증 중
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

### 파인 튜닝된 모델 활용한 챗봇


In [11]:
print("새로운 모델의 이름은: " + str(fine_tune_job))

새로운 모델의 이름은: FineTuningJob(id='ftjob-4vXvYpLi37N0sGykyVbKAJQS', created_at=1734619976, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AgCl24td', finished_at=1734622178, hyperparameters=Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-ujuF1J8GkXR8ggNiF9BMBXJp', result_files=['file-5ixixtUmfNqZUdXmYiLstg'], seed=640049167, status='succeeded', trained_tokens=1574688, training_file='file-MF66BN6rvWdktNsqWqov34', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 2, 'learning_rate_multiplier': 1.8}}})


In [15]:
# app.py

# 실행 환경이 구글 코랩의 경우, 인증 부분은 앞에서 실행했다면, 작성하지 않아도 됩니다.
# 인증 부분 init_api()

import click

# 파인튜닝된 모델 이름 설정
# model = "$FINE_TUNED_MODEL"
model = "ft:gpt-4o-mini-2024-07-18:personal::AgCl24td"


# 시스템 프롬프트 설정
system_prompt = (
    "당신은 MendMind, AI 정신 건강 코치입니다. "
    "기본적으로 한글로 물어보면 한글로 답변을 해 주렴."
    "당신의 목적은 공감, 이해, 그리고 감정적·심리적 어려움을 "
    "관리하는 통찰력으로 사용자의 정신 건강 여정을 지원하는 것입니다. "
    "일반적인 조언과 정서적 지원을 제공할 수 있지만, 개인적인 연락, "
    "약속 일정 잡기, 또는 특정 위치 세부 정보를 공유할 수는 없습니다. "
    "당신의 유일한 역할은 사용자에게 대처 전략을 제공하고, 정신 건강 "
    "주제에 대한 정보를 제공하며, 필요한 경우 전문적인 자원을 안내하는 것입니다. "
    "사용자와 일반적인 대화를 할 수 있지만, 당신의 주요 초점은 "
    "기밀성과 배려로 사용자의 웰빙을 위한 길을 지원하는 것입니다."
)

# 기본 메시지 설정
base_messages = [
    {
        "role": "system",
        "content": system_prompt
    },
    {
        "role": "user",
        "content": "안녕하세요."
    },
    {
        "role": "assistant",
        "content": "저는 MendMind입니다. "
                   "AI 정신 건강 코치입니다. "
                   "오늘 어떻게 도와드릴까요?"
    }
]

# 인터랙티브 세션 시작
while True:
    # 기본 메시지를 복사하여 사용
    messages = base_messages.copy()

    # 사용자 입력 받기
    request = input(
        click.style(
            "입력: (종료하려면 'exit' 또는 'quit' 입력): ",
            fg="green"
        )
    )

    # 'exit' 또는 'quit' 입력 시 종료
    if request.lower() in ["exit", "quit"]:
        break

    # 사용자 입력을 메시지에 추가
    messages.append(
        {
            "role": "user",
            "content": f"{request}"
        }
    )

    # API에 메시지 전송
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,  # 출력의 창의성 조절
        frequency_penalty=0.5,  # 반복 억제
        presence_penalty=0.5,  # 새로운 주제 언급 유도
    )

    # 응답 내용 가져오기
    content = response.choices[0].message.content.strip()

    # 출력 결과를 보기 좋게 표시
    click.echo(
        click.style(
            "출력: ",
            fg="yellow"
        ) + content
    )
    click.echo()  # 새로운 줄 추가


입력: (종료하려면 'exit' 또는 'quit' 입력): 나는 심리적 상담을 하고 싶어. 가능할까?
출력: 안녕하세요. 미국에서 심리상담사는 licensed therapist로 보통 많이 불려집니다. 이 Licensed라는 단어가 붙은 이유는 대부분 주(State)에서 규정한 시험을 통과해야만 상담이 가능하다는 뜻입니다. 그러므로 주에 상관없이 해당주에서 Licensed된 therapist와 상담을 하셔야 합니다. 만약 한국의 어떤 therapist가 미국의 licensed therapist라고 주장한다면, 그분은 반드시 미국에서 Licensed된 therapist여야 하고 또한 해당주에서 상담이 가능해야합니다. 위에 말씀드린 것처럼 주마다 규정과 시험이 다르기 때문에 한국분이 미국 Licensed Therapist와 상담하는 것은 법적인 문제가 생길 수 있습니다. 그렇지만 한국의 licensed counselor와 상담하는 것은 아무런 문제가 되지 않습니다. 북한과 가까운 지역에는 licensed counselor들이 활동하고 계십니다. 한국에서도 licensed counselor들이 활동할 수 있도록 허용하고 있으니 시간이 지나면 한국에도 licensed counselor들이 생기겠네요?

입력: (종료하려면 'exit' 또는 'quit' 입력): 어떻게 해야 모든 사람에게 제가 무가치하다는 느낌을 바꿀 수 있을까요?
출력: 안녕하세요. 기미를 느끼시고, 무가치하다는 생각이 드신다구요? 우선, 자신을 소중하게 여기셔야하고, 스스로 기미도 느껴보시고, 잘하신 점도 인식해 보세요. 남들과 비교하지 마시구요. 모든사람에게 가치있는 분이라는 것을 인식시키는 방법은 여러분들이 다가가는 방법도 있겠지만 그 보다 더 중요한 것은 여러분들이 자신의 가치를 먼저 알고 표현하는것이 아닐까 싶습니다.

입력: (종료하려면 'exit' 또는 'quit' 입력): 저는 심각한 우울증과 불안감에 시달리고 있습니다. 정말 많은 일이 벌어지고 있는 것 같고, 이로 인해 너무 산만해져

### 20-3. 모델의 한계: 실제 애플리케이션에서 발생할 수 있는 한계